## Installs

In [ ]:
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [1]:
#!pip install datasets

## Imports

In [ ]:
import pandas as pd

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz

In [ ]:
import torch
import torch.nn as nn

In [ ]:
import re

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

In [ ]:
import os 
directory = "gradients"
parent_dir = "results"
path = os.path.join(parent_dir, directory) 

## Import Model

In [ ]:
import sys
sys.path.append('model/code-bert/')
from temporal_relation_classification import TemporalRelationClassification
from temporal_relation_classification_config import TemporalRelationClassificationConfig

In [ ]:
model_path = "saved_models/bert-base-uncased-saved-model"
model = TemporalRelationClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30526, 768)

## Initialise Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model.to(device)

TemporalRelationClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30526, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
model.eval()

TemporalRelationClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30526, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

## Import Dataset

In [119]:
original = pd.read_csv('data/annotated/original.csv')
counterfactuals = pd.read_csv('data/annotated/counterfactuals.csv')

## Compute Gradients Original

In [138]:
import re
def annotate_text(row, column):
    context = row['context']
    eventA = re.escape(row['eventA'])
    eventB = re.escape(row['eventB'])

    # Make sure the longer event is replaced first if they overlap
    if len(eventA) > len(eventB):
        context = re.sub(eventA, f"[a1]{row['eventA']}[/a1]", context)
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
    else:
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
        context = re.sub(eventA, f"[a1]{row['eventB']}[/a1]", context)

    return context

In [139]:
original['annotated_context'] = original.apply(lambda row: annotate_text(row, 'context'), axis=1)

In [140]:
from datasets import Dataset, DatasetDict
dataset = DatasetDict({
    "original": Dataset.from_pandas(original),
})

In [141]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenizer_class = str(type(tokenizer)).strip("><'").split('.')[-1]

In [142]:
def preprocess_function(examples):
  max_length_value = 508
  return tokenizer(examples["annotated_context"], truncation=True, max_length=max_length_value)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [145]:
def predict(inputs, token_type_ids):
    output = model(inputs, token_type_ids=token_type_ids)
    logits = output[0]
    max_logits = logits.max(dim=1).values
    return max_logits

In [146]:
def create_baseline(input_ids, attention_mask):
    baseline_attention_mask = torch.ones_like(attention_mask).to(device)
    baseline_input_ids = torch.zeros_like(input_ids).to(device)

    special_token_ids = {30522, 30523, 30524, 30525}
    special_tokens_mask = torch.isin(input_ids, torch.tensor(list(special_token_ids), device=device))
    baseline_input_ids[special_tokens_mask] = input_ids[special_tokens_mask]
    return baseline_input_ids, baseline_attention_mask #baseline_token_type_ids

In [147]:
lig = LayerIntegratedGradients(predict, model.bert.embeddings)

In [148]:
def gradient_sensitivity(model, input_ids, attention_mask):
    model.eval()
    baseline_input_ids, baseline_attention_mask = create_baseline(input_ids, attention_mask)

    attributions, delta = lig.attribute(inputs=(input_ids, attention_mask),
                                         baselines=(baseline_input_ids, baseline_attention_mask),
                                         n_steps=10,  
                                         return_convergence_delta=True)

    attributions = attributions.sum(dim=-1).squeeze(0)  
    attributions = attributions / torch.norm(attributions) 

    return attributions  

In [149]:
def tokenisation(text, tokenizer, device='cuda'):
    special_tokens_dict = {'additional_special_tokens': ['[a1]', '[/a1]', '[a2]', '[/a2]']}
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    tokenizer.add_special_tokens(special_tokens_dict)

    encoded_input = tokenizer(text, max_length=508, truncation=True, padding='max_length', return_tensors='pt')
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input.get('attention_mask', None).to(device) if encoded_input.get('attention_mask', None) is not None else None
    token_type_ids = encoded_input.get('token_type_ids', None).to(device) if encoded_input.get('token_type_ids', None) is not None else None
    return input_ids, attention_mask, token_type_ids

In [150]:
gradients_details = []
for index, row in original.iterrows():
    print(index)
    text = row['annotated_context']
    input_ids, attention_mask, token_type_ids = tokenisation(text, tokenizer, device)
    attributions = gradient_sensitivity(model, input_ids, attention_mask)
    attributions_list = attributions.detach().cpu().numpy().tolist()
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    word_attributions = list(zip(tokens, attributions_list))  # Ensure this pairs tokens with their attributions correctly

    gradients_details.append({
        "index": index,
        "text": text,
        "word_attributions": word_attributions
    })


0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


In [151]:
gradients_df = pd.DataFrame(gradients_details)
print(gradients_df)

     index                                               text  \
0        0  Israeli Prime Minister Benjamin Netanyahu apol...   
1        1  Israeli Prime Minister Benjamin Netanyahu apol...   
2        2  Israeli Prime Minister Benjamin Netanyahu apol...   
3        3  Israeli Prime Minister Benjamin Netanyahu apol...   
4        4  Israeli Prime Minister Benjamin Netanyahu apol...   
..     ...                                                ...   
191    191  The FAA on Friday announced it will close 149 ...   
192    192  The flu season is winding down, and it has kil...   
193    193  Our digital age is all about bits, those preci...   
194    194  A Cyprus exit from the euro union, if it comes...   
195    195  Israel's prime minister has apologised to Turk...   

                                     word_attributions  
0    [([CLS], 0.003622958006703595), (israeli, 0.00...  
1    [([CLS], -0.07436703143894202), (israeli, 0.00...  
2    [([CLS], 0.06415117608911086), (israeli, -0

In [152]:
original['index'] = original.index
gradients_df['index'] =  gradients_df.index
original = original.reset_index(drop=True)
gradients_df['label'] = df_filtered['label']

In [154]:
gradients_df.to_csv('results/gradients/gradients-og-bert-base.csv', index=False)

## Compute Gradients Counterfactuals

In [155]:
import re
def annotate_text(row, column):
    context = row['counterfactual']
    eventA = re.escape(row['eventA'])
    eventB = re.escape(row['eventB'])

    # Make sure the longer event is replaced first if they overlap
    if len(eventA) > len(eventB):
        context = re.sub(eventA, f"[a1]{row['eventA']}[/a1]", context)
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
    else:
        context = re.sub(eventB, f"[a2]{row['eventB']}[/a2]", context)
        context = re.sub(eventA, f"[a1]{row['eventB']}[/a1]", context)

    return context

In [156]:
counterfactuals['annotated_context'] = counterfactuals.apply(lambda row: annotate_text(row, 'counterfactual'), axis=1)

In [157]:
from datasets import Dataset, DatasetDict
dataset = DatasetDict({
    "counterfactual": Dataset.from_pandas(df_filtered_counterfactuals),
})

In [158]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenizer_class = str(type(tokenizer)).strip("><'").split('.')[-1]

In [159]:
def preprocess_function(examples):
  max_length_value = 508
  return tokenizer(examples["annotated_context"], truncation=True, max_length=max_length_value)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

In [162]:
def predict(inputs, token_type_ids):
    output = model(inputs, token_type_ids=token_type_ids)
    logits = output[0]
    max_logits = logits.max(dim=1).values
    return max_logits

In [163]:
def create_baseline(input_ids, attention_mask):
    baseline_attention_mask = torch.ones_like(attention_mask).to(device)
    #baseline_token_type_ids = torch.zeros_like(token_type_ids).to(device)
    baseline_input_ids = torch.zeros_like(input_ids).to(device)

    special_token_ids = {30522, 30523, 30524, 30525}
    special_tokens_mask = torch.isin(input_ids, torch.tensor(list(special_token_ids), device=device))
    baseline_input_ids[special_tokens_mask] = input_ids[special_tokens_mask]
    return baseline_input_ids, baseline_attention_mask #baseline_token_type_ids

In [164]:
lig = LayerIntegratedGradients(predict, model.bert.embeddings)

In [165]:
def gradient_sensitivity(model, input_ids, attention_mask):
    model.eval()
    baseline_input_ids, baseline_attention_mask = create_baseline(input_ids, attention_mask)

    attributions, delta = lig.attribute(inputs=(input_ids, attention_mask),
                                         baselines=(baseline_input_ids, baseline_attention_mask),
                                         n_steps=10,  # More steps for smoother integration
                                         return_convergence_delta=True)
    # Sum over the embedding dimension and normalize
    attributions = attributions.sum(dim=-1).squeeze(0)  # Ensure it reduces to the number of tokens
    attributions = attributions / torch.norm(attributions)  # Normalization

    return attributions  # Convert to list for easier processing later

In [166]:
def tokenisation(text, tokenizer, device='cuda'):
    special_tokens_dict = {'additional_special_tokens': ['[a1]', '[/a1]', '[a2]', '[/a2]']}
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    tokenizer.add_special_tokens(special_tokens_dict)

    encoded_input = tokenizer(text, max_length=508, truncation=True, padding='max_length', return_tensors='pt')
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input.get('attention_mask', None).to(device) if encoded_input.get('attention_mask', None) is not None else None
    token_type_ids = encoded_input.get('token_type_ids', None).to(device) if encoded_input.get('token_type_ids', None) is not None else None
    return input_ids, attention_mask, token_type_ids

In [ ]:
gradients_details_counterfactuals = []
for index, row in counterfactuals.iterrows():
    print(index)
    text = row['annotated_context']
    input_ids, attention_mask, token_type_ids = tokenisation(text, tokenizer, device)
    attributions = gradient_sensitivity(model, input_ids, attention_mask)
    attributions_list = attributions.detach().cpu().numpy().tolist()
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    word_attributions = list(zip(tokens, attributions_list))  # Ensure this pairs tokens with their attributions correctly

    gradients_details_counterfactuals.append({
        "index": index,
        "text": text,
        "word_attributions": word_attributions
    })


0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


In [ ]:
gradients_counterfactual_df = pd.DataFrame(gradients_details_counterfactuals)
print(gradients_counterfactual_df)

In [ ]:
counterfactuals['index'] = counterfactuals.index
gradients_counterfactual_df['index'] =  gradients_counterfactual_df.index
counterfactuals = counterfactuals.reset_index(drop=True)

In [ ]:
gradients_counterfactual_df['label'] = df_counter['label']

In [ ]:
gradients_counterfactual_df.to_csv('results/gradients/gradients-cf-bert-base.csv', index=False)

## Visualise Instance

In [ ]:
from IPython.display import HTML, display
import pandas as pd

instance = gradients_df.loc[1, 'word_attributions']
tokens, raw_attributions = zip(*instance)
attributions = [ __builtins__.sum(attrs) for attrs in raw_attributions] #/len(attrs)

def visualize_attributions(tokens, attributions):
    attributions = [float(i) for i in attributions] 
    min_attribution = min(attributions)
    max_attribution = max(attributions)
    attributions = [(attr - min_attribution) / (max_attribution - min_attribution) for attr in attributions]

    html_string = "<p><b>Attributions:</b><br>"
    for token, attr in zip(tokens, attributions):
        color = f"{int(300 * (1 - attr))}" 
        html_string += f"<span style='background-color:rgb(255,{color},{color})'>{token}</span> "

    html_string += "</p>"
    display(HTML(html_string))

visualize_attributions(tokens, attributions)